In [ ]:
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D, AveragePooling2D
from keras.models import Model
from glob import glob
import re
import os.path
import scipy.misc
import numpy as np
import random
import matplotlib.pyplot as plt
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import keras.backend as K

image_shape = (320,480)
data_dir = './data_mio'
namefile = 'grieta'
batch_size = 3

In [ ]:
def generator(data_folder, image_shape, batch_size):
    """  Create batches of training and validation data """
    image_paths = glob(os.path.join(data_folder, 'images', '*.png'))
    label_paths = {
        re.sub(r'_road_', '_', os.path.basename(path)): path
        for path in glob(os.path.join(data_folder, 'labels', '*_road_*.png'))}
    
    background_color = np.array([255, 0, 255])  ## Solo el magenta

    random.shuffle(image_paths)
    while 1: 
        for batch_i in range(0, len(image_paths), batch_size):
            images = []
            gt_images = []
            for image_file in image_paths[batch_i:batch_i+batch_size]:
                gt_image_file = label_paths[os.path.basename(image_file)]
                image = scipy.misc.imread(image_file)
                gt_image = scipy.misc.imread(gt_image_file)
                #image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)
                #gt_image = scipy.misc.imresize(scipy.misc.imread(gt_image_file), image_shape)
                gt_bg = np.all(gt_image == background_color, axis=2)  #Binarizar la segmentación del target
                gt_bg = gt_bg.reshape(*gt_bg.shape, 1)   # Añadirle una dimensión al tensor
                gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)  #crear dos capas reflejo
                images.append(image)
                gt_images.append(gt_image)
                
            X = np.array(images)
            Y = np.array(gt_images)
            yield (X, Y)
        
train_generator = generator(os.path.join(data_dir, 'training'), image_shape, batch_size)
validation_generator = generator(os.path.join(data_dir, 'validation'), image_shape, batch_size)
test_generator = generator(os.path.join(data_dir, 'test'), image_shape, batch_size)

In [ ]:
def f1 (y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    tp = K.sum(y_true_f * y_pred_f)        # suma de una AND   son los True Points
    fp = K.sum(K.clip(K.clip(y_pred_f+y_true_f,0,1)-y_true_f,0,1))
    fn = K.sum(K.clip(K.clip(y_pred_f+y_true_f,0,1)-y_pred_f,0,1))
    Pr = tp/(tp+fp)
    Re = tp/(tp+fn)
    f1t = 2*(Pr*Re)/(Pr+Re)
    return f1t

In [ ]:
# U-Net-A model

def unet(img_rows, img_cols, img_channels):  # 23 trainable layers, use same padding, en lugar de unpadding layers
    x = Input(shape=(img_rows, img_cols, img_channels))
    
    # Encoder 
    conv1 = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    conv2 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
    #pool1 = AveragePooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(128, (3, 3), padding='same', activation='relu')(pool1)
    conv4 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv3)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv4)
    #pool2 = AveragePooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = Conv2D(256, (3, 3), padding='same', activation='relu')(pool2)
    conv6 = Conv2D(256, (3, 3), padding='same', activation='relu')(conv5)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv6)
    #pool3 = AveragePooling2D(pool_size=(2, 2))(conv6)
    
    conv7 = Conv2D(512, (3, 3), padding='same', activation='relu')(pool3)
    conv8 = Conv2D(512, (3, 3), padding='same', activation='relu')(conv7)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv8)
    #pool4 = AveragePooling2D(pool_size=(2, 2))(conv8)
    
    conv9 = Conv2D(1024, (3, 3), padding='same', activation='relu')(pool4)
    conv10 = Conv2D(1024, (3, 3), padding='same', activation='relu')(conv9)
    
    # Decoder
    convT1 = Conv2D(512, (2, 2), padding='same', activation='relu')(UpSampling2D(size=(2, 2))(conv10))
    merge1 = concatenate([convT1, conv8], axis=3)
    conv11 = Conv2D(512, (3, 3), padding='same', activation='relu')(merge1)
    conv12 = Conv2D(512, (3, 3), padding='same', activation='relu')(conv11)
    
    convT2 = Conv2D(256, (2, 2), padding='same', activation='relu')(UpSampling2D(size=(2, 2))(conv12))
    merge2 = concatenate([convT2, conv6], axis=3)
    conv13 = Conv2D(256, (3, 3), padding='same', activation='relu')(merge2)
    conv14 = Conv2D(256, (3, 3), padding='same', activation='relu')(conv13)
    
    convT3 = Conv2D(128, (2, 2), padding='same', activation='relu')(UpSampling2D(size=(2, 2))(conv14))
    merge3 = concatenate([convT3, conv4], axis=3)
    conv15 = Conv2D(128, (3, 3), padding='same', activation='relu')(merge3)
    conv16 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv15)
    
    convT4 = Conv2D(64, (2, 2), padding='same', activation='relu')(UpSampling2D(size=(2, 2))(conv16))
    merge4 = concatenate([convT4, conv2], axis=3)
    conv17 = Conv2D(64, (3, 3), padding='same', activation='relu')(merge4)
    conv18 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv17)

    y = Conv2D(2, (1, 1), activation='softmax')(conv18)

    return Model(inputs=x, outputs=y)

In [ ]:
# U-Net-B Model

def unetB(img_rows, img_cols, img_channels):
    x = Input(shape=(img_rows, img_cols, img_channels))
    
    conv1 = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    conv2 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(128, (3, 3), padding='same', activation='relu')(pool1)
    conv4 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv3)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = Conv2D(256, (3, 3), padding='same', activation='relu')(pool2)
    conv6 = Conv2D(256, (3, 3), padding='same', activation='relu')(conv5)
    
    
    convT3 = Conv2D(128, (2, 2), padding='same', activation='relu')(UpSampling2D(size=(2, 2))(conv6))
    merge3 = concatenate([convT3, conv4], axis=3)
    conv15 = Conv2D(128, (3, 3), padding='same', activation='relu')(merge3)
    conv16 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv15)
    
    convT4 = Conv2D(64, (2, 2), padding='same', activation='relu')(UpSampling2D(size=(2, 2))(conv16))
    merge4 = concatenate([convT4, conv2], axis=3)
    conv17 = Conv2D(64, (3, 3), padding='same', activation='relu')(merge4)
    conv18 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv17)
    
    
    y = Conv2D(2, (1, 1), activation='softmax')(conv18)

    return Model(inputs=x, outputs=y)

In [ ]:
# U-Net-C model

def unetC(img_rows, img_cols, img_channels):  #lr = 0.0009
    x = Input(shape=(img_rows, img_cols, img_channels))
    
    # Encoder 
    conv1 = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    conv2 = Conv2D(64, (1, 1), padding='same', activation='relu')(conv1)
    pool = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    
    conv3 = Conv2D(128, (3, 3), padding='same', activation='relu')(pool)
    conv4 = Conv2D(128, (1, 1), padding='same', activation='relu')(conv3)
    
    # Decoder   
    convT = Conv2D(64, (2, 2), padding='same', activation='relu')(UpSampling2D(size=(2, 2))(conv4))
    merge = concatenate([convT, conv2], axis=3)
    conv5 = Conv2D(64, (3, 3), padding='same', activation='relu')(merge)
    conv6 = Conv2D(64, (1, 1), padding='same', activation='relu')(conv5)
    
    # Segmentation
    y = Conv2D(2, (1, 1), activation='softmax')(conv6)

    return Model(inputs=x, outputs=y)

In [ ]:
model = unetB(image_shape[0],image_shape[1],3)
adam = optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) # lr = 0.0009
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=[f1])
checkpointer = ModelCheckpoint(filepath='bestmodel_' + namefile + '.h5', monitor='val_loss', verbose=0, save_best_only=True,
                              mode='min', period=1)

h = model.fit_generator(generator = train_generator, steps_per_epoch=200//batch_size, epochs=5, verbose=1, callbacks=[checkpointer], 
                        validation_data=validation_generator, validation_steps=18//batch_size)

In [ ]:
model.load_weights('bestmodel_grieta_unetb1' + '.h5')


score_train = model.evaluate_generator(train_generator, steps=18//batch_size)
print('training', score_train)
score_val = model.evaluate_generator(validation_generator, steps=18//batch_size)
print('validation', score_val)
score_test = model.evaluate_generator(test_generator, steps=38//batch_size)
print('test', score_test)

In [ ]:
import time

plt.rcParams['figure.figsize'] = (15,15) 

data_folder = './resultados/aiglern/' 
th_segment = 0.5
k = 10
filelist = glob(os.path.join(data_folder, '*.png'))

i = 1

inicio = time.time()

for image_file in filelist[0:k]:
    img_original = scipy.misc.imread(image_file)
    img = scipy.misc.imresize(img_original, image_shape) 
    y = model.predict(np.expand_dims(img, axis=0), batch_size=1)   
    mask = np.stack([y[0,:,:,0] > th_segment,np.zeros(image_shape),np.zeros(image_shape)], axis=2)
    masked = np.ma.masked_array(mask, img).astype('float32')
    z = str(i).zfill(3)
    i += 1
    
fin = time.time()
print ((fin-inicio)/10)


data_folder = './resultados/uneta/aiglern/'
k = 10
filelist = glob(os.path.join(data_folder, '*.png'))
i = 1
f1t = 0
prt = 0
ret = 0
for image_file in filelist[0:k]:
    img = scipy.misc.imread(image_file)
    img = np.asarray(img)
    a = scipy.misc.imread('./resultados/labels_aiglern/'+str(i).zfill(3)+'b.png')  
    b = np.empty([len(a),len(a[0])])
    for m in range (len(a)):
        for n in range (len (a[0])):
            if a[m][n][0] == 0:
                b[m][n] = 0
            else:
                b[m][n] = 1

    c = np.empty([len(img),len(img[0])])
    for m in range (len(img)):
        for n in range (len (img[0])):
            if img[m][n][0] == 0:
                c[m][n] = 0
            else:
                c[m][n] = 1

    tp = 0
    fp = 0
    fn = 0
    for r in range(len(c)):
        for o in range(len(c[0])):
            if c[r][o] == 1 and b[r-5:r+5,o-5:o+5].sum() >= 1:
                tp += 1
            elif c[r][o] == 1 and b[r][o] == 0:
                fp += 1
            elif c[r-5:r+5,o-5:o+5].sum() == 0 and b[r][o] == 1:
                fn += 1
    pr = tp/(tp+fp)
    re = tp/(tp+fn)
    f1 = 2*(pr*re)/(pr+re)
    print (i,' Pr: ',pr, 'Re: ', re, 'F1: ', f1)
    
    f1t = (f1t+f1)
    prt = prt+pr
    ret = ret+re
    
    i += 1

print (' PrT: ', prt/10, ' ReT: ', ret/10, 'F1T: ', f1t/10)